Load Libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from geopy.extra.rate_limiter import RateLimiter

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium --yes
import folium # map rendering library

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\VYudin\Anaconda3

  added / updated specs:
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.10.1              |             py_0          59 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         112 KB

The following NEW packages will be INSTALLED:

  branca             conda-forge/noarch::branca-0.3.1-py_0
  folium             conda-forge/noarch::folium-0.10.1-py_0
  vincent            conda-forge/noarch::vincent-0.4.4-py_1




branca-0.3.1         | 25 KB     |            |   0% 
branca-0.3.1         | 25 KB     | ######3    |  63% 
b

Load information about national capitals from Wiki:

In [4]:
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_national_capitals_by_population',header=0)[1]

And see first 5:

In [5]:
df.head()

,Rank,Country/Territory,Capital,Population,Year,% ofcountry'spopulation
0,1,China PR,Beijing,"21,542,000[1]",2010,1.5%
1,2,Japan,Tokyo,"13,929,286[2]",2017,11.03%
2,3,Russia,Moscow,"12,506,468[3]",2011,8.52%
3,4,DR Congo,Kinshasa,"11,855,000[4]",2012,12.9%
4,5,Indonesia,Jakarta,"10,075,310[5]",2011,3.76%


In some rows we can see two cities or city name + something. We can't take coordinates with wrong name city, so will check and correct it. In 155 row we have two "Populations", so correct it too:

In [7]:
chg=[(57,"Guayaquil"),(81,"Guatemala"),(89,"Naypyidaw"),(100,"Jerusalem"),
     (154,"El Aaiún"),(163,"Berne"),(171,"Nicosia"),(200,"Ramallah"),(231,"Yaren"),(235,"Vatican City"),(239,"Brades")]

In [8]:
for i in range(len(chg)):
    df.loc[chg[i][0],"Capital"]=chg[i][1]
df.loc[154,"Population"]="194668"

In [9]:
df.head()

,Rank,Country/Territory,Capital,Population,Year,% ofcountry'spopulation
0,1,China PR,Beijing,"21,542,000[1]",2010,1.5%
1,2,Japan,Tokyo,"13,929,286[2]",2017,11.03%
2,3,Russia,Moscow,"12,506,468[3]",2011,8.52%
3,4,DR Congo,Kinshasa,"11,855,000[4]",2012,12.9%
4,5,Indonesia,Jakarta,"10,075,310[5]",2011,3.76%


Creating two DataFrame with capital's coordinates and if we take Error in taking coordinate:

In [10]:
coord=pd.DataFrame(columns=["Capital","Latitude","Longitude"])
err=pd.DataFrame(columns=["Number","Name"])

And take coordinate from Nomutatim. When we take coordinate, we print result for viewing progress:

In [11]:
for i in df.index:
    geolocator = Nominatim(user_agent="tr_explorer")
    address =df.loc[i,"Capital"]    
    try:
        location = geolocator.geocode(address,timeout=1)
        latitude = location.latitude
        longitude = location.longitude        
    except:
        coord=coord.append({"Capital":address,"Latitude":np.nan,"Longitude":np.nan},ignore_index=True)
        err=err.append({"Number":i,"Name":address},ignore_index=True)
        print('The geograpical coordinate of {} denied'.format(address))
    else:
        coord=coord.append({"Capital":address,"Latitude":latitude,"Longitude":longitude},ignore_index=True)
        print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Beijing are 39.906217, 116.3912757.
The geograpical coordinate of Tokyo are 35.6828387, 139.7594549.
The geograpical coordinate of Moscow are 55.4792046, 37.3273304.
The geograpical coordinate of Kinshasa are -4.3217055, 15.3125974.
The geograpical coordinate of Jakarta are -6.1753942, 106.827183.
The geograpical coordinate of Seoul are 37.564982549999996, 126.93921080358436.
The geograpical coordinate of Cairo are 30.048819, 31.243666.
The geograpical coordinate of Mexico City are 19.4326296, -99.1331785.
The geograpical coordinate of Dhaka are 23.7593572, 90.3788136.
The geograpical coordinate of London are 51.5073219, -0.1276474.
The geograpical coordinate of Lima are -12.0621065, -77.0365256.
The geograpical coordinate of Tehran are 35.7006177, 51.4013785.
The geograpical coordinate of Bangkok are 13.7542529, 100.493087.
The geograpical coordinate of Hanoi are 21.0294498, 105.8544441.
The geograpical coordinate of Riyadh are 24.6319692, 46.7150648.
The

The geograpical coordinate of Macau are 22.1757605, 113.5514142.
The geograpical coordinate of Edinburgh are 55.9533456, -3.1883749.
The geograpical coordinate of Djibouti (city) are 11.8145966, 42.8453061.
The geograpical coordinate of Yamoussoukro are 6.809107, -5.273263.
The geograpical coordinate of Bissau are 11.861324, -15.583055.
The geograpical coordinate of Tallinn are 59.4372155, 24.7453688.
The geograpical coordinate of Bratislava are 48.1516988, 17.1093063.
The geograpical coordinate of San Juan denied
The geograpical coordinate of Canberra are -35.2975906, 149.1012676.
The geograpical coordinate of Wellington are -41.2887953, 174.7772114.
The geograpical coordinate of Sarajevo are 43.8519774, 18.3866868.
The geograpical coordinate of Juba are 4.8472017, 31.5951655.
The geograpical coordinate of Cardiff are 51.4816546, -3.1791934.
The geograpical coordinate of Belfast are 54.5964411, -5.9302761.
The geograpical coordinate of San José are 37.3361905, -121.8905833.
The geogra

Will see Errors:

In [13]:
err

,Number,Name
0,66,Sofia
1,75,Yerevan
2,85,Stockholm
3,93,Bishkek
4,103,Tirana
5,112,Asmara
6,123,San Salvador
7,133,San Juan
8,142,Vientiane
9,151,Gaborone


If we saw error with name of Capital, we add it in our list "chg" upper. Try again take cooridinates, and if we success delete row from DataFrame err:

In [14]:
for i in err.index:
    geolocator = Nominatim(user_agent="tr_explorer")
    address =err.loc[i,"Name"]
    try:
        location = geolocator.geocode(address,timeout=1)
        latitude = location.latitude
        longitude = location.longitude        
    except:
        print('The geograpical coordinate of {} denied'.format(address))
    else:
        coord.loc[err.loc[i,"Number"],"Latitude"]=latitude
        coord.loc[err.loc[i,"Number"],"Longitude"]=longitude
        err=err.drop(i,axis=0)
        print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Sofia are 42.6978634, 23.3221789.
The geograpical coordinate of Yerevan are 40.1776121, 44.5125849.
The geograpical coordinate of Stockholm are 59.3251172, 18.0710935.
The geograpical coordinate of Bishkek are 42.8765615, 74.6070079.
The geograpical coordinate of Tirana are 41.31588575, 19.900912164461147.
The geograpical coordinate of Asmara are 15.3389667, 38.9326763.
The geograpical coordinate of San Salvador are 13.6989939, -89.1914249.
The geograpical coordinate of San Juan are 18.465299, -66.116666.
The geograpical coordinate of Vientiane are 17.9640988, 102.6133707.
The geograpical coordinate of Gaborone are -24.6581357, 25.9088474.
The geograpical coordinate of Manama are 26.2235041, 50.5822436.
The geograpical coordinate of Gustavia are 17.8957043, -62.8508372.
The geograpical coordinate of Longyearbyen are 78.2231558, 15.6463656.
The geograpical coordinate of Brades are 16.7890601, -62.208731.


Look Error again:

In [15]:
err

,Number,Name


In [13]:
    address ="Vatican City"
    geolocator = Nominatim(user_agent="yudin_vd")
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))
    except:
        print('The geograpical coordinate of {} denied'.format(address))

The geograpical coordinate of Vatican City are 41.90381485, 12.453152716779822.


It's empty now. If we take error again we try and try to all succesful. Look to first 5 rows in DataFrame coord:

In [16]:
coord.head()

,Capital,Latitude,Longitude
0,Beijing,39.906217,116.391276
1,Tokyo,35.682839,139.759455
2,Moscow,55.479205,37.327330
3,Kinshasa,-4.321706,15.312597
4,Jakarta,-6.175394,106.827183


Add columns Latitude and Longitude in our main DataFrame and see result

In [17]:
df["Latitude"]=coord["Latitude"]
df["Longitude"]=coord["Longitude"]

In [18]:
df.head()

,Rank,Country/Territory,Capital,Population,Year,% ofcountry'spopulation,Latitude,Longitude
0,1,China PR,Beijing,"21,542,000[1]",2010,1.5%,39.906217,116.391276
1,2,Japan,Tokyo,"13,929,286[2]",2017,11.03%,35.682839,139.759455
2,3,Russia,Moscow,"12,506,468[3]",2011,8.52%,55.479205,37.327330
3,4,DR Congo,Kinshasa,"11,855,000[4]",2012,12.9%,-4.321706,15.312597
4,5,Indonesia,Jakarta,"10,075,310[5]",2011,3.76%,-6.175394,106.827183


Check lists for Errors:

In [19]:
for i in coord.index:
    if coord.loc[i,"Capital"]!=df.loc[i,"Capital"]:
        print("Error in {} row".format(i))

Check types in our DataFrame:

In [22]:
df.dtypes

Rank                        object
Country/Territory           object
Capital                     object
Population                  object
Year                        object
% ofcountry'spopulation     object
Latitude                   float64
Longitude                  float64
dtype: object

Correct Population to integer and drop column "Rank"

In [23]:
for i in df.index:
    b=df.loc[i,"Population"].find("[")
    if b!=-1:
        df.loc[i,"Population"]=df.loc[i,"Population"][:b]
    b=df.loc[i,"Population"].find("(")
    if b!=-1:
        df.loc[i,"Population"]=df.loc[i,"Population"][:b-1]
    df.loc[i,"Population"]=df.loc[i,"Population"].replace(",","")
        

In [24]:
df[["Population"]]=df[["Population"]].astype("int")

In [25]:
df=df.drop("Rank",axis=1)

See result:

In [26]:
df.head()

,Country/Territory,Capital,Population,Year,% ofcountry'spopulation,Latitude,Longitude
0,China PR,Beijing,21542000,2010,1.5%,39.906217,116.391276
1,Japan,Tokyo,13929286,2017,11.03%,35.682839,139.759455
2,Russia,Moscow,12506468,2011,8.52%,55.479205,37.327330
3,DR Congo,Kinshasa,11855000,2012,12.9%,-4.321706,15.312597
4,Indonesia,Jakarta,10075310,2011,3.76%,-6.175394,106.827183


Because every Capital have different size and we have only 99,500 Regular Calls/Day, we take some count of venue for every city based by Population:

In [28]:
df["CountVenue"]=df["Population"]//100000*5

In [29]:
for i in df.index:
    if df.loc[i,"Population"]<2000000:
        df.loc[i,"CountVenue"]=100

In [30]:
df.head(10)

,Country/Territory,Capital,Population,Year,% ofcountry'spopulation,Latitude,Longitude,CountVenue
0,China PR,Beijing,21542000,2010,1.5%,39.906217,116.391276,1075
1,Japan,Tokyo,13929286,2017,11.03%,35.682839,139.759455,695
2,Russia,Moscow,12506468,2011,8.52%,55.479205,37.327330,625
3,DR Congo,Kinshasa,11855000,2012,12.9%,-4.321706,15.312597,590
4,Indonesia,Jakarta,10075310,2011,3.76%,-6.175394,106.827183,500
5,South Korea,Seoul,9838892,2015,19.03%,37.564983,126.939211,490
6,Egypt,Cairo,9500000,2012,9.54%,30.048819,31.243666,475
7,Mexico,Mexico City,8918653,2015,7.05%,19.432630,-99.133178,445
8,Bangladesh,Dhaka,8906039,2011,5.52%,23.759357,90.378814,445
9,United Kingdom England,London,8908081,2015,13.19%,51.507322,-0.127647,445


Checked sum of Count:

In [32]:
df["CountVenue"].sum()

32880

And save our result in local file, becase from Nominatim Usage Policy we must cached Results or we would be blocked:

In [33]:
df.to_excel("capitals_2.xlsx", index=False)